## 加载数据

In [12]:
from feature_selection import FeatureSelection


def convert_ranking(arr):
    n = len(arr)
    arr_revers = np.zeros(n, dtype=int)
    for rank in range(n):
        feature_index = arr[rank]
        arr_revers[feature_index] = rank  # 假设排名从0开始
    return arr_revers


# 数据集参数
Gisette = {'DATASETNAME': 'gisette.mat'}
Isolet = {'DATASETNAME': 'Isolet.mat'}
Madelon = {'DATASETNAME': 'madelon.mat'}
Wine = {'DATASETNAME': 'Wine.mat'}
GLIOMA = {'DATASETNAME': 'GLIOMA.mat'}
Lung = {'DATASETNAME': 'lung.mat'}
Ovarian = {'DATASETNAME': 'Ovarian.mat'}
Datasets = [Gisette, Isolet, Madelon, Wine, GLIOMA, Lung, Ovarian]

fs = FeatureSelection(Datasets)
fs.pre_process(Datasets[6], random_state=42)
fs.display_distribution()

trainset distribution: [113  64]
testset distribution: [49 27]
number of feature: 15154


## 特征选择
- 应用不同的特征选择算法
- 将选择的特征转换成索引

In [ ]:
import numpy as np
from skfeature.function.similarity_based import fisher_score
from skfeature.function.information_theoretical_based import CIFE
from skfeature.function.statistical_based import chi_square

idx_1 = fisher_score.fisher_score(fs.x_train, fs.y_train, mode='index')
idx_1_convert = convert_ranking(idx_1)
# print(idx_1)
print(idx_1_convert)

idx_2 = chi_square.chi_square(fs.x_train, fs.y_train, mode='index')
idx_2_convert = convert_ranking(idx_2)
# print(idx_2)
print(idx_2_convert)

idx_3 = CIFE.cife(fs.x_train, fs.y_train, mode='index', n_selected_features=fs.x_train.shape[1])
idx_3_convert = convert_ranking(idx_3)
# print(idx_3)
print(idx_3_convert)

[5638 5333 4524 ... 9425 9431 9430]
[ 6539  5511  4700 ... 10054 10055 10056]


## 非支配排序

In [ ]:
def non_dominated_sort(arr1, arr2, arr3):
    """
    对三个特征排名数组进行非支配排序，返回所有前沿
    参数:
        arr1, arr2, arr3: 一维ndarray数组，表示三个不同的特征排名 
    返回:
        list of lists: 每个子列表代表一个前沿面，按Front 1, Front 2, ...排列
    """
    # 1. 合并三个数组的排名
    num_features = len(arr1)
    features = np.column_stack((arr1, arr2, arr3))
    # 2. 非支配排序
    domination_counts = np.zeros(num_features, dtype=int)
    dominated_features = [[] for _ in range(num_features)]
    fronts = [[]]  # fronts[0] = Front 1, fronts[1] = Front 2, ...
    # 计算支配关系
    for i in range(num_features):
        for j in range(i + 1, num_features):
            # 检查i是否支配j
            if np.all(features[i] <= features[j]) and np.any(features[i] < features[j]):
                dominated_features[i].append(j)
                domination_counts[j] += 1
    # 初始化第一前沿面（Front 1）
    fronts[0] = [i for i in range(num_features) if domination_counts[i] == 0]
    # 构建后续前沿面
    current_front = 0
    while current_front < len(fronts) and fronts[current_front]:
        next_front = []
        for i in fronts[current_front]:
            for j in dominated_features[i]:
                domination_counts[j] -= 1
                if domination_counts[j] == 0:
                    next_front.append(j)
        current_front += 1
        if next_front:
            fronts.append(next_front)
    # 3. 返回所有前沿（每个前沿内的特征按原始索引升序排列）
    return [sorted(front) for front in fronts]
# 示例用法
# if __name__ == "__main__":
#     # 示例数据
#     arr1 = np.array([1, 3, 2, 5, 4])  # 算法1的排名（越小越好）
#     arr2 = np.array([2, 1, 3, 4, 5])  # 算法2的排名
#     arr3 = np.array([1, 2, 4, 3, 5])  # 算法3的排名
#     # 0、1、
#     all_fronts = non_dominated_sort(arr1, arr2, arr3)
#     print("所有Pareto前沿：")
#     for i, front in enumerate(all_fronts, 1):
#         print(f"Front {i}: {front}")
all_fronts = non_dominated_sort(idx_1_convert, idx_2_convert, idx_3_convert)
print(f"Front 1: {all_fronts[0]}")

## 前后对比

In [ ]:
from sklearn.base import clone
from sklearn.neural_network import MLPClassifier
from metrics import calculate_gmean_mauc_f1
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore")  # 忽略警告

def train_and_test(model, x_train, x_test, y_train, y_test):
    scaler = StandardScaler()  # 数据的标准化
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    model.fit(x_train, y_train)  # 模型训练
    y_test_pred_proba = model.predict_proba(x_test)
    gmean, mauc, f1 = calculate_gmean_mauc_f1(y_test_pred_proba, y_test)  # 计算准确率指标
    return gmean, mauc, f1

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(10,), max_iter=100, random_state=42)
gmean, mauc, f1 = train_and_test(clone(model), fs.x_train, fs.x_test, fs.y_train, fs.y_test)
print(f"gmean: {gmean}, mauc: {mauc}, f1: {f1}")

In [ ]:
x_train, y_train = SMOTE(random_state=42, k_neighbors=4).fit_resample(fs.x_train, fs.y_train)
gmean, mauc, f1 = train_and_test(clone(model), x_train, fs.x_test, y_train, fs.y_test)
print(f"gmean: {gmean}, mauc: {mauc}, f1: {f1}")

In [ ]:
x_train, y_train = SMOTE(random_state=42, k_neighbors=4).fit_resample(fs.x_train[:, all_fronts[0]], fs.y_train)
gmean, mauc, f1 = train_and_test(clone(model), x_train, fs.x_test[:, all_fronts[0]], y_train, fs.y_test)
print(f"gmean: {gmean}, mauc: {mauc}, f1: {f1}")